In [17]:
import av
import numpy as np
import io
import edge_tts


def mp3_to_pcm_bytes_fast(mp3_input, sample_rate=16000):
    if isinstance(mp3_input, (bytes, bytearray)):
        mp3_input = io.BytesIO(mp3_input)

    container = av.open(mp3_input)
    stream = container.streams.audio[0]

    resampler = av.audio.resampler.AudioResampler(
        format="s16", layout="mono", rate=sample_rate
    )

    pcm_data = bytearray()

    for frame in container.decode(stream):
        frames = resampler.resample(frame)

        if isinstance(frames, list):
            for f in frames:
                pcm_data.extend(bytes(f.planes[0]))
        else:
            pcm_data.extend(bytes(frames.planes[0]))

    container.close()
    return bytes(pcm_data)


text = """
Trong “Chuyện người con gái Nam Xương”, chi tiết cái bóng là một hình tượng nghệ thuật đặc sắc
"""
communicate = edge_tts.Communicate(text, "vi-VN-HoaiMyNeural")


async for chunk in communicate.stream():
    mp3_buf = io.BytesIO()
    if chunk["type"] == "audio":
        print(len(mp3_to_pcm_bytes_fast(chunk["data"])))

4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
4416
1728


In [ ]:


mp3_buf.seek(0)
audio = AudioSegment.from_file(mp3_buf, format="mp3")
pcm = audio.set_frame_rate(16000).set_channels(1).set_sample_width(2)
pcm_bytes = pcm.raw_data

print(len(pcm_bytes))

170496


In [9]:
import asyncio
import wave
import edge_tts
import subprocess

# ---- Đường dẫn ffmpeg ----
# Nếu ffmpeg đã có trong PATH, chỉ cần "ffmpeg"
# Nếu bạn để trong dự án (Windows)
FFMPEG_PATH = r"src/model/mp3-to-pcm/ffmpeg.exe"
# Nếu chạy Linux/Render -> dùng bản static (ví dụ "src/bin/ffmpeg")


async def tts_to_wav_stream(text, wav_path="output.wav", sample_rate=16000):
    """
    Dùng Edge-TTS tạo âm thanh, stream qua ffmpeg để decode MP3 -> PCM,
    và ghi PCM vào file WAV theo thời gian thực (mượt, không giật)
    """

    # --- tạo tiến trình ffmpeg, đọc ghi qua pipe ---
    process = await asyncio.create_subprocess_exec(
        FFMPEG_PATH,
        "-hide_banner",
        "-loglevel",
        "error",
        "-i",
        "pipe:0",  # input MP3 qua stdin
        "-ac",
        "1",  # mono
        "-ar",
        str(sample_rate),  # sample rate
        "-f",
        "s16le",  # PCM 16-bit
        "pipe:1",  # output PCM qua stdout
        stdin=asyncio.subprocess.PIPE,
        stdout=asyncio.subprocess.PIPE,
    )

    # --- khởi tạo edge-tts ---
    communicate = edge_tts.Communicate(text, "vi-VN-HoaiMyNeural")

    # --- mở file WAV ---
    with wave.open(wav_path, "wb") as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)  # 16-bit PCM
        wf.setframerate(sample_rate)

        # --- task 1: feed MP3 chunks vào ffmpeg ---
        async def feed_mp3():
            async for chunk in communicate.stream():
                if chunk["type"] == "audio":
                    process.stdin.write(chunk["data"])
                    await process.stdin.drain()
            process.stdin.close()

        # --- task 2: đọc PCM output từ ffmpeg ---
        async def read_pcm():
            while True:
                pcm = await process.stdout.read(4096)
                if not pcm:
                    break
                wf.writeframes(pcm)

        # --- chạy song song ---
        await asyncio.gather(feed_mp3(), read_pcm())

    print(f"✅ Saved smooth WAV file: {wav_path}")


# ---- chạy thử ----
text = """
Bạn có muốn mình mở rộng thêm phần phát audio trực tiếp (nghe ngay khi đang TTS) thay vì chỉ lưu file không
"""

await tts_to_wav_stream(text)

NotImplementedError: 

In [2]:
import os

print(os.getcwd())

e:\Project\chatbot_esp32\esp32_server
